## Quick Guide to Performance and Parallel Programming

There are many options available to improve the performance of your Python codes.
The first thing to determine is what is limiting your computation. It could be CPU
speed (unlikely), memory limitations (out-of-core computing), or it could be data
transfer speed (waiting on data to arrive for processing). If your code is pure-Python,
then you can try running it with Pypy, which is is an alternative Python implementa-
tion that employs a just-in-time compiler. If your code does not experience a massive
speed-up with Pypy, then there is probably something external to the code that is
slowing it down (e.g., disk access or network access). If Pypy doesn’t make any
sense because you are using many compiled modules that Pypy does not support,
then there are many diagnostic tools available

Python has its own built-in profiler cProfile you can invoke from the command
line as in the following


```bash
python -m cProfile -o program.prof my_program.py
```

### Testing with the Fibonacci sequence

In [30]:
def fib(n):
    """Display the n first terms of Fibonacci sequence"""
    a, b = 0, 1
    i = 0
    fib_list = []
    fib_list.append(b)
    while i < n:
        a, b = b, a+b
        fib_list.append(b)
        i +=1
    print(fib_list[0])

In [63]:
%%timeit
fib(100000)

1
1
1
1
1
1
1
1
236 ms ± 9.39 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [65]:
236*10e-3

2.36

In [66]:
%prun fib(100000)

1
 

         100040 function calls in 0.428 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.336    0.336    0.405    0.405 <ipython-input-30-6bb8f8030fb8>:1(fib)
   100001    0.069    0.000    0.069    0.000 {method 'append' of 'list' objects}
        1    0.022    0.022    0.428    0.428 <string>:1(<module>)
        1    0.000    0.000    0.428    0.428 {built-in method builtins.exec}
        3    0.000    0.000    0.000    0.000 socket.py:334(send)
        3    0.000    0.000    0.000    0.000 iostream.py:195(schedule)
        2    0.000    0.000    0.000    0.000 iostream.py:366(write)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.print}
        3    0.000    0.000    0.000    0.000 threading.py:1062(_wait_for_tstate_lock)
        3    0.000    0.000    0.000    0.000 threading.py:1104(is_alive)
        2    0.000    0.000    0.000    0.000 iostream.py:300(_is_master_process)
        

In [67]:
236*10e-3/0.428

5.5140186915887845

## multiprocessing

Python has a multiprocessing module that is part of the standard library. This makes it easy
to spawn child worker processes that can break off and individually process small
parts of a big job. However, it is still your responsibility as the programmer to figure
out how to distribute the data for your algorithm.

In [1]:
def fib(n):
    """Display the n first terms of Fibonacci sequence"""
    a, b = 0, 1
    i = 0
    fib_list = []
    fib_list.append(b)
    while i < n:
        a, b = b, a+b
        fib_list.append(b)
        i +=1
    return(fib_list[0])

In [79]:
%time test = [fib(i) for i in range(100000,100020,1)]

CPU times: user 2.31 s, sys: 12 ms, total: 2.32 s
Wall time: 2.32 s


In [81]:
from multiprocessing import Pool

In [82]:
pool = Pool()

In [85]:
%time test2 = pool.map(fib, range(100000,100020,1))

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 563 ms


In [91]:
pool.terminate()

## dask


In [92]:
!pip install dash

    100% |████████████████████████████████| 36.9MB 700kB/s 
    100% |████████████████████████████████| 163kB 1.7MB/s 
  Running setup.py bdist_wheel for dash ... done
  Stored in directory: /home/hvribeiro/.cache/pip/wheels/06/af/07/6f112f3eb4d01b90e817d728799be80f36398567368aef27e3
  Running setup.py bdist_wheel for flask-compress ... done
  Stored in directory: /home/hvribeiro/.cache/pip/wheels/96/32/88/a1f6d9dd3c29570ab3a8acc0d556b3b20abcf3c623c868ce0a
  Running setup.py bdist_wheel for dash-renderer ... done
  Stored in directory: /home/hvribeiro/.cache/pip/wheels/4b/dc/d0/e1ae0332df5fb1f056153818831750032936535150fed856b5
  Running setup.py bdist_wheel for retrying ... done
  Stored in directory: /home/hvribeiro/.cache/pip/wheels/d7/a9/33/acc7b709e2a35caa7d4cae442f6fe6fbf2c43f80823d46460c
Successfully built dash flask-compress dash-renderer retrying
twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
python-louvain 0.6 has requirement networkx<2.0.0, but you'll hav

In [2]:
%%time
test3 = []
for i in range(100000,100020,1):
    a = fib(i)
    test3.append(a)
test3

CPU times: user 2.67 s, sys: 2.15 s, total: 4.82 s
Wall time: 4.82 s


In [1]:
import dask
from dask.distributed import Client, progress
client = Client()

In [17]:
test4 = []
for i in range(100000,100020,1):
    a = dask.delayed(fib)(i)
    test4.append(a)
res = dask.delayed()(test4)

In [18]:
res

Delayed('list-8a6a7a5e-d403-4308-b8d5-535e7c96ce54')

open http://localhost:8787/status to see the progress

In [14]:
%time res.compute()

CPU times: user 368 ms, sys: 12 ms, total: 380 ms
Wall time: 1.14 s


[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [19]:
res = res.persist()  # start computation in the background

In [21]:
res.compute()

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]